<h1 style="font-size: 60px; text-align: center;">
Numerical tiling-based simulations of (re)decoherence<br>
in single & multi-field models of inflation
</h1>


In [ ]:
# ============================================================
# 1. General style
# ============================================================
from modules_py.styles_format import configure_style, sci_notation
configure_style()
# ============================================================
# 2. General architecture
# ============================================================
from modules_py.architecture import create_split_file
# ============================================================
# 3. General templates
# ============================================================
from modules_py.generate_fortran import load_template, build_parameter_sets, generate_fortran_files
# ============================================================
# 4. General runner
# ============================================================
from modules_py.run_fortran import run_simulation
# ============================================================
# 5. General loader
# ============================================================
from modules_py.load_data import load_data, get_ylabel_dict, load_structure
# ============================================================
# 6. Accidents
# ============================================================
from modules_py.configurations import make_split_incs, preview_accidents, generate_rectangular_grid
# ============================================================
# Plots
# ============================================================
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib notebook
# ============================================================

<h1 style="font-size: 40px; text-align: center;">
Single-field models of inflation
</h1>

In [ ]:
"""
Initialization of the single-field case:
creation of Fortran files, loading of the template, and
preparation of the data structure.
"""

create_split_file(mode="single") 
template_single = load_template("fortran_template_single.txt") 
data_case_single = {}

In [ ]:
"""
Optional: conversion of an image into a normalized grid (bitmap) to generate a custom structure in the simulation.
"""

img = Image.open("Images_Accidents/landscape.png")
img_gray = img.convert("L")
img_resize = img_gray.resize((48, 48), Image.NEAREST)
matrix_resume = np.array(img_resize)
matrix_resume_norm = ((matrix_resume / 255))

intIM = generate_rectangular_grid(x_divisions=48,
                                  y_divisions=48,
                                  x_limits=(6, 70),
                                  y_limits=(5, 10),
                                  norm_matrix=matrix_resume_norm)

In [ ]:
"""
Settings, single-field:
free case, manual accidents, or image-defined environment.

It is possible to redefine the simulation parameters/settings 
(dangerous: first review the modules and their respective documentation).
"""

configurations_single = {
    # Caso default: mod_pref = 0
    0: {
        "case_custom": {
            "fortran_mod_pref": 0,
            "fortran_mod_accident": 1
        },
        
        "accidents": [] #Caso default Free accidents
    },
    
    # Case 1: mod_pref = 1
    1: {
        "case_custom": {
            "fortran_mod_pref": 1,
            "fortran_mod_accident": 0
        },
        
        "accidents": [(-0.05, 3.0, 20.0, 2.5, 7.0, 1.0),
        ]
    },

    # Case 2: mod_pref = 2
    2: {
        "case_custom": {
            "fortran_mod_pref": 2,
            "fortran_mod_accident": 1
        },
        
        "accidents": intIM
    },

    # More...
}

In [ ]:
"""
Selection of the base case (without environment/accidents)
Important: The case without environment is not optional
it will be used to optimize the case with environment.
"""

case_single_id = 0

"""
Construction of the Fortran parameter set based on the chosen configuration.
This step unifies case, global, and parallel overrides.
"""

cfg = configurations_single[case_single_id]

param_sets_single = build_parameter_sets(
    mode="single",
    case_overrides=cfg.get("case_custom"),
    global_overrides=cfg.get("global_custom"),
    parallel_overrides=cfg.get("parallel_custom")
)

"""
Generation of .f90 files from the template and defined parameters
"""

generated_single = generate_fortran_files(template_single, "single", param_sets_single)

In [ ]:
"""
Compilation and execution of Fortran codes
Each part is executed in a separate screen session.
"""

run_simulation("single", param_sets_single)

In [ ]:
# Save numerical output:

data_case_single[case_single_id] = load_structure("single", param_sets_single)

In [ ]:
# =============================================================================
# FIRST PART: POWER DATA
# =============================================================================

# Base configuration
save_power_spectrum_referential = False
saved_power_spectrum_referential_figures = []
reference_magnitude = 'zeta'

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
ax.set_yscale('log')

ax.plot(
    np.log(data_case_single[0]["power_spectrum"]["kvar"]),
    data_case_single[0]["power_spectrum"][reference_magnitude],
    color='black',
    linewidth=2.0,
    linestyle='-',
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("single")

plt.xlabel(ylabel_dict['kvar'] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)

plt.grid(True, which="both", ls="--", alpha=0.5)
ax.legend()

# Saving
if save_power_spectrum_referential:
    base_name = f"Power_Spectrum_{fortran_mod_pref:03d}"
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_power_spectrum_referential_figures.append(fig)

plt.show()

In [ ]:
# =============================================================================
# SECOND PART: ELLIPSE DATA
# =============================================================================

# Base configuration
save_reference_ellipse = False
saved_reference_ellipse_figures = []
reference_magnitude = 'Determinant'
reference_ellipse_number = 20

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
# ax.set_yscale('log')

ax.plot(
    data_case_single[0]["ellipse"][reference_ellipse_number]["N"],
    data_case_single[0]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='-', color='black', lw=2.0, alpha=0.7,
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("single")


plt.xlabel(ylabel_dict['N'], fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude], fontsize=16)
plt.grid(True, which="both", ls="--", alpha=0.5)

ax.legend()

# Saving
if save_reference_ellipse:
    base_name = (
        f"Ellipse_{reference_ellipse_number}_case_"
        f"{fortran_mod_pref:03d}_{reference_magnitude}"
    )
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_reference_ellipse_figures.append(fig)

plt.show()


In [ ]:
# Reference line used to calibrate the optimal construction of accidents.

reference_line_y_single = np.log(1.0/data_case_single[0]["power_spectrum"]["kphys"][0])
print(reference_line_y_single)

# Generation of accidents

make_split_incs(configurations_single, "single", param_sets_single, reference_line_y_single)

In [ ]:
# Graphic preview of accidents

figs_pre_single = preview_accidents(
    configurations_single,
    "single",
    param_sets_single,
    reference_line_y_single,
    data_case_single[0],
)

for fig in figs_pre_single:
    fig.show()

In [ ]:
# Case of a particular environment

case_single_id = 1


cfg = configurations_single[case_single_id]

param_sets_single = build_parameter_sets(
    mode="single",
    case_overrides=cfg.get("case_custom"),
    global_overrides=cfg.get("global_custom"),
    parallel_overrides=cfg.get("parallel_custom")
)

generated_single = generate_fortran_files(template_single, "single", param_sets_single)

In [ ]:
run_simulation("single", param_sets_single)

In [ ]:
data_case_single[case_single_id] = load_structure("single", param_sets_single)

In [ ]:
# =============================================================================
# FIRST PART: POWER DATA
# =============================================================================

# Base configuration
save_power_spectrum_referential = False
saved_power_spectrum_referential_figures = []
reference_magnitude = 'zeta'

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
ax.set_yscale('log')

ax.plot(
    np.log(data_case_single[case_single_id]["power_spectrum"]["kvar"]),
    data_case_single[case_single_id]["power_spectrum"][reference_magnitude],
    color='black',
    linewidth=2.0,
    linestyle='-',
    label='Without environment'
)

ax.plot(
    np.log(data_case_single[0]["power_spectrum"]["kvar"]),
    data_case_single[0]["power_spectrum"][reference_magnitude],
    color='red',
    linewidth=2.0,
    linestyle='--',
    label='With environment'
)


ylabel_dict = get_ylabel_dict("single")

plt.xlabel(ylabel_dict['kvar'] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)

plt.grid(True, which="both", ls="--", alpha=0.5)
ax.legend()

# Saving
if save_power_spectrum_referential:
    base_name = f"Power_Spectrum_{fortran_mod_pref:03d}"
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_power_spectrum_referential_figures.append(fig)

plt.show()

In [ ]:
# =============================================================================
# SECOND PART: ELLIPSE DATA
# =============================================================================

# Base configuration
save_reference_ellipse = False
saved_reference_ellipse_figures = []
reference_magnitude = 'Determinant'
reference_ellipse_number = 50

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
# ax.set_yscale('log')

ax.plot(
    data_case_single[case_single_id]["ellipse"][reference_ellipse_number]["N"],
    data_case_single[case_single_id]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='-', color='black', lw=2.0, alpha=0.7,
    label='With environment'
)

ax.plot(
    data_case_single[0]["ellipse"][reference_ellipse_number]["N"],
    data_case_single[0]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='--', color='red', lw=2.0, alpha=0.7,
    label='Without environment'
)


ylabel_dict = get_ylabel_dict("single")


plt.xlabel(ylabel_dict['N'], fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude], fontsize=16)
plt.grid(True, which="both", ls="--", alpha=0.5)

ax.legend()

# Saving
if save_reference_ellipse:
    base_name = (
        f"Ellipse_{reference_ellipse_number}_case_"
        f"{fortran_mod_pref:03d}_{reference_magnitude}"
    )
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_reference_ellipse_figures.append(fig)

plt.show()

<h1 style="font-size: 40px; text-align: center;">
Two-field models of inflation
</h1>

In [ ]:
"""
Initialization of the two-field case:
creation of Fortran files, loading of the template, and
preparation of the data structure.
"""

create_split_file(mode="two_field") 
template_two_field = load_template("fortran_template_two_field.txt") 
data_case_two_field = {}

In [ ]:
"""
Settings, two-field:
free case, manual accidents, or image-defined environment.

It is possible to redefine the simulation parameters/settings 
(dangerous: first review the modules and their respective documentation).
"""

configurations_two_field = {
    # Caso default: mod_pref = 0
    0: {
        "case_custom": {
            "fortran_mod_pref": 0,
            "fortran_mod_accident": 1
        },
        
        "accidents": [] #Caso default Free accidents
    },
    
    # Case 1: mod_pref = 1
    1: {
        "case_custom": {
            "fortran_mod_pref": 1,
            "fortran_mod_accident": 0
        },
        
        "global_custom": {            
            #Isocurvature
            "fortran_interaction_two_fields":"I⊥ = α₁ v⊥₁ + α₂ v⊥₂",
            "fortran_alpha_1_two_fields":1.0,
            "fortran_alpha_2_two_fields":1.0,
            "fortran_interaction_1_two_fields": "alpha(1) - ( alpha(1)*phi_dot(1)*phi_dot(1) + alpha(2)*phi_dot(1)*phi_dot(2) )/( phi_dot(1)*phi_dot(1) + phi_dot(2)*phi_dot(2) )",
            "fortran_interaction_2_two_fields": "alpha(2) - ( alpha(1)*phi_dot(1)*phi_dot(2) + alpha(2)*phi_dot(2)*phi_dot(2) )/( phi_dot(1)*phi_dot(1) + phi_dot(2)*phi_dot(2) )",
        },
        
        "accidents": [(0.46, 3.0, 10.0, 2.5, 7.0, 1.0),
                      (-0.05, 3.0, 20.0, 2.5, 7.0, 1.0),
        ]
    },

    # Case 2: mod_pref = 2
    2: {
        "case_custom": {
            "fortran_mod_pref": 2,
            "fortran_mod_accident": 1
        },
        
        "accidents": intIM
    },

    # More...
}

In [ ]:
"""
Selection of the base case (without environment/accidents)
Important: The case without environment is not optional
it will be used to optimize the case with environment.
"""

case_two_field_id = 0

"""
Construction of the Fortran parameter set based on the chosen configuration.
This step unifies case, global, and parallel overrides.
"""

cfg = configurations_two_field[case_two_field_id]

param_sets_two_field = build_parameter_sets(
    mode="two_field",
    case_overrides=cfg.get("case_custom"),
    global_overrides=cfg.get("global_custom"),
    parallel_overrides=cfg.get("parallel_custom")
)

"""
Generation of .f90 files from the template and defined parameters
"""

generated_two_field = generate_fortran_files(template_two_field, "two_field", param_sets_two_field)

In [ ]:
"""
Compilation and execution of Fortran codes
Each part is executed in a separate screen session.
"""

run_simulation("two_field", param_sets_two_field)

In [ ]:
# Save numerical output:

data_case_two_field[case_two_field_id] = load_structure("two_field", param_sets_two_field)

In [ ]:
# =============================================================================
# FIRST PART: POWER DATA
# =============================================================================

# Base configuration
save_power_spectrum_referential = False
saved_power_spectrum_referential_figures = []
reference_magnitude = 'zeta'

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
ax.set_yscale('log')

ax.plot(
    np.log(data_case_two_field[0]["power_spectrum"]["kvar"]),
    data_case_two_field[0]["power_spectrum"][reference_magnitude],
    color='black',
    linewidth=2.0,
    linestyle='-',
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("two_field")

plt.xlabel(ylabel_dict['kvar'] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)

plt.grid(True, which="both", ls="--", alpha=0.5)
ax.legend()

# Saving
if save_power_spectrum_referential:
    base_name = f"Power_Spectrum_{fortran_mod_pref:03d}"
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_power_spectrum_referential_figures.append(fig)

plt.show()

In [ ]:
# =============================================================================
# SECOND PART: ELLIPSE DATA
# =============================================================================

# Base configuration
save_reference_ellipse = False
saved_reference_ellipse_figures = []
reference_magnitude = 'Determinant'
reference_ellipse_number = 20

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
# ax.set_yscale('log')

ax.plot(
    data_case_two_field[0]["ellipse"][reference_ellipse_number]["N"],
    data_case_two_field[0]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='-', color='black', lw=2.0, alpha=0.7,
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("two_field")


plt.xlabel(ylabel_dict['N'], fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude], fontsize=16)
plt.grid(True, which="both", ls="--", alpha=0.5)

ax.legend()

# Saving
if save_reference_ellipse:
    base_name = (
        f"Ellipse_{reference_ellipse_number}_case_"
        f"{fortran_mod_pref:03d}_{reference_magnitude}"
    )
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_reference_ellipse_figures.append(fig)

plt.show()

In [ ]:
# Reference line used to calibrate the optimal construction of accidents.

reference_line_y_two_field = np.log(1.0/data_case_two_field[0]["power_spectrum"]["kphys"][0])
print(reference_line_y_two_field)

# Generation of accidents

make_split_incs(configurations_two_field, "two_field", param_sets_two_field, reference_line_y_two_field)

In [ ]:
# Graphic preview of accidents

figs_pre_two_field = preview_accidents(
    configurations_two_field,
    "two_field",
    param_sets_two_field,
    reference_line_y_two_field,
    data_case_two_field[0],
)

for fig in figs_pre_two_field:
    fig.show()

In [ ]:
# Case of a particular environment

case_two_field_id = 1

cfg = configurations_two_field[case_two_field_id]

param_sets_two_field = build_parameter_sets(
    mode="two_field",
    case_overrides=cfg.get("case_custom"),
    global_overrides=cfg.get("global_custom"),
    parallel_overrides=cfg.get("parallel_custom")
)

generated_two_field = generate_fortran_files(template_two_field, "two_field", param_sets_two_field)


In [ ]:
run_simulation("two_field", param_sets_two_field)

In [ ]:
data_case_two_field[case_two_field_id] = load_structure("two_field", param_sets_two_field)

In [ ]:
# =============================================================================
# FIRST PART: POWER DATA
# =============================================================================

# Base configuration
save_power_spectrum_referential = False
saved_power_spectrum_referential_figures = []
reference_magnitude = 'zeta'

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
ax.set_yscale('log')

ax.plot(
    np.log(data_case_two_field[case_two_field_id]["power_spectrum"]["kvar"]),
    data_case_two_field[case_two_field_id]["power_spectrum"][reference_magnitude],
    color='black',
    linewidth=2.0,
    linestyle='-',
    label='With environment'
)

ax.plot(
    np.log(data_case_two_field[0]["power_spectrum"]["kvar"]),
    data_case_two_field[0]["power_spectrum"][reference_magnitude],
    color='red',
    linewidth=2.0,
    linestyle='--',
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("two_field")

plt.xlabel(ylabel_dict['kvar'] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude] + '$\\displaystyle{ \\Bigg|_{\\epsilon=1} }$', fontsize=16)

plt.grid(True, which="both", ls="--", alpha=0.5)
ax.legend()

# Saving
if save_power_spectrum_referential:
    base_name = f"Power_Spectrum_{fortran_mod_pref:03d}"
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_power_spectrum_referential_figures.append(fig)

plt.show()

In [ ]:
# =============================================================================
# SECOND PART: ELLIPSE DATA
# =============================================================================

# Base configuration
save_reference_ellipse = False
saved_reference_ellipse_figures = []
reference_magnitude = 'Determinant'
reference_ellipse_number = 50

fig, ax = plt.subplots(figsize=(6, 4), constrained_layout=True)
# ax.set_yscale('log')

ax.plot(
    data_case_two_field[case_two_field_id]["ellipse"][reference_ellipse_number]["N"],
    data_case_two_field[case_two_field_id]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='-', color='black', lw=2.0, alpha=0.7,
    label='With environment'
)

ax.plot(
    data_case_two_field[0]["ellipse"][reference_ellipse_number]["N"],
    data_case_two_field[0]["ellipse"][reference_ellipse_number][reference_magnitude],
    linestyle='--', color='red', lw=2.0, alpha=0.7,
    label='Without environment'
)

ylabel_dict = get_ylabel_dict("two_field")


plt.xlabel(ylabel_dict['N'], fontsize=16)
plt.ylabel(ylabel_dict[reference_magnitude], fontsize=16)
plt.grid(True, which="both", ls="--", alpha=0.5)

ax.legend()

# Saving
if save_reference_ellipse:
    base_name = (
        f"Ellipse_{reference_ellipse_number}_case_"
        f"{fortran_mod_pref:03d}_{reference_magnitude}"
    )
    for fmt in ['pdf']:
        fig.savefig(f"{base_name}.{fmt}", dpi=500, bbox_inches='tight')

saved_reference_ellipse_figures.append(fig)

plt.show()